In [1]:
import numpy as np
from pygsti.baseobjs import Basis
from itertools import product


# Commutator Helper Functions
def commute(mat1, mat2):
    return mat1 @ mat2 + mat2 @ mat1


def anti_commute(mat1, mat2):
    return mat1 @ mat2 - mat2 @ mat1

def hamiltonian_error_generator(initial_state, indexed_pauli, identity):
    return (
        -1j * indexed_pauli @ initial_state @ identity
        + 1j * identity @ initial_state @ indexed_pauli
    )

# Convert basis
def convert_to_pauli(matrix, numQubits):
    pauliNames1Q = ["I", "X", "Y", "Z"]
    # Hard force to 1- or 2-qubit
    if numQubits == 1:
        pauliNames = pauliNames1Q
    elif numQubits == 2:
        pauliNames = ["".join(name) for name in product(pauliNames1Q, pauliNames1Q)]
    translationMatrix = pp.from_std_transform_matrix
    coefs = np.real_if_close(np.dot(translationMatrix, matrix.flatten()))
    return [(a, b) for (a, b) in zip(coefs, pauliNames) if abs(a) > 0.0001]

np.set_printoptions(precision=1, linewidth=1000)

c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\baseobjs\opcalc\__init__.py:31: UserWarning: 
An optimized Cython-based implementation of `pygsti.baseobjs.opcalc` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

  _warnings.warn(warn_msg)
c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\circuits\circuitparser\__init__.py:60: UserWarning: 
An optimized Cython-based implementation of `pygsti.circuits.circuitparser` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

 

In [3]:
numQubits = 1
pp1Q = Basis.cast("PP", dim=4)
pp = Basis.cast("PP", dim=4**numQubits)
pauliNames1Q = ["I", "X", "Y", "Z"]
pauliStates1Q = ["X+", "X-", "Y+", "Y-", "Z+", "Z-"]
# Temporary Hard Coding Error Gens to 1-, and 2-qubits
if numQubits == 1:
    pauliNames = pauliNames1Q
    initialStates = pauliStates1Q
hamiltonianIndices = pauliNames[1:]
hamiltonianErrorOutputs = dict()
for index in hamiltonianIndices:
    for state in initialStates:
        if state[-1] == "-":
            mat1 = -1 * pp1Q[state[0]]
        elif state[-1] == "+":
            mat1 = pp1Q[state[0][0]]
        else:
            mat1 == pp1Q["I"]
        inputState = mat1
        ident = pp1Q["I"]
        inputState = ident / 2 + inputState / 2
        tempIndexed = pp1Q[index]
        hamiltonianErrorOutputs[(index, state)] = hamiltonian_error_generator(
                inputState, tempIndexed, ident
            )
for key in hamiltonianErrorOutputs:
    hamiltonianErrorOutputs[key] = convert_to_pauli(
        hamiltonianErrorOutputs[key], numQubits
    )
print(hamiltonianErrorOutputs)

{('X', 'X+'): [], ('X', 'X-'): [], ('X', 'Y+'): [(1.0, 'Z')], ('X', 'Y-'): [(-1.0, 'Z')], ('X', 'Z+'): [(-1.0, 'Y')], ('X', 'Z-'): [(1.0, 'Y')], ('Y', 'X+'): [(-1.0, 'Z')], ('Y', 'X-'): [(1.0, 'Z')], ('Y', 'Y+'): [], ('Y', 'Y-'): [], ('Y', 'Z+'): [(1.0, 'X')], ('Y', 'Z-'): [(-1.0, 'X')], ('Z', 'X+'): [(1.0, 'Y')], ('Z', 'X-'): [(-1.0, 'Y')], ('Z', 'Y+'): [(-1.0, 'X')], ('Z', 'Y-'): [(1.0, 'X')], ('Z', 'Z+'): [], ('Z', 'Z-'): []}


In [8]:
np.kron(pp["X"], pp["X"])

array([[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [9]:
numQubits = 2
pp1Q = Basis.cast("PP", dim=4)
pp = Basis.cast("PP", dim=4**numQubits)
pauliNames1Q = ["I", "X", "Y", "Z"]
pauliStates1Q = ["X+", "X-", "Y+", "Y-", "Z+", "Z-"]
# Temporary Hard Coding Error Gens to 1-, and 2-qubits
if numQubits == 1:
    pauliNames = pauliNames1Q
    initialStates = pauliStates1Q
elif numQubits == 2:
    pauliNames = ["".join(name) for name in product(pauliNames1Q, pauliNames1Q)]
    initialStates = [
        ",".join((name))
        for name in product(pauliStates1Q + ["I"], pauliStates1Q + ["I"])
    ][:-1]


# Compute the set of measurable effects of Hamiltonian error generators operating on two qubits in each of the specified eigenstates
hamiltonianIndices = pauliNames[1:]
hamiltonianErrorOutputs = dict()

for index in hamiltonianIndices:
    for state in initialStates:
        tempState = dict(enumerate(state.split(",")))
        tempIndex = dict(enumerate(index))
        print(tempIndex)
        print(tempState)

        if numQubits == 2:
            if tempState[0][-1] == "-":
                mat1 = -1 * pp1Q[tempState[0][0]]
            elif tempState[0][-1] == "+":
                mat1 = pp1Q[tempState[0][0]]
            else:
                mat1 == pp1Q["I"]
            if tempState[1][-1] == "-":
                mat2 = -1 * pp1Q[tempState[1][0]]
            elif tempState[1][-1] == "+":
                mat2 = pp1Q[tempState[1][0]]
            else:
                mat2 == pp1Q["I"]
            ident = pp["II"]

            inputState = np.kron(mat1, mat2)
            inputState = ident / 2 + inputState / 2
            # print(inputState)
            # print(pp[index])
            tempIndexed = np.kron(pp1Q[tempIndex[0]], pp1Q[tempIndex[1]])
            # print(tempIndexed)
            # print(hamiltonian_error_generator(inputState, tempIndexed, ident))
            # sys.exit()
            hamiltonianErrorOutputs[(index, state)] = hamiltonian_error_generator(
                inputState, tempIndexed, ident
            )
for key in hamiltonianErrorOutputs:
    hamiltonianErrorOutputs[key] = convert_to_pauli(
        hamiltonianErrorOutputs[key], numQubits
    )

print(hamiltonianErrorOutputs)

{0: 'I', 1: 'X'}
{0: 'X+', 1: 'X+'}
{0: 'I', 1: 'X'}
{0: 'X+', 1: 'X-'}
{0: 'I', 1: 'X'}
{0: 'X+', 1: 'Y+'}
{0: 'I', 1: 'X'}
{0: 'X+', 1: 'Y-'}
{0: 'I', 1: 'X'}
{0: 'X+', 1: 'Z+'}
{0: 'I', 1: 'X'}
{0: 'X+', 1: 'Z-'}
{0: 'I', 1: 'X'}
{0: 'X+', 1: 'I'}
{0: 'I', 1: 'X'}
{0: 'X-', 1: 'X+'}
{0: 'I', 1: 'X'}
{0: 'X-', 1: 'X-'}
{0: 'I', 1: 'X'}
{0: 'X-', 1: 'Y+'}
{0: 'I', 1: 'X'}
{0: 'X-', 1: 'Y-'}
{0: 'I', 1: 'X'}
{0: 'X-', 1: 'Z+'}
{0: 'I', 1: 'X'}
{0: 'X-', 1: 'Z-'}
{0: 'I', 1: 'X'}
{0: 'X-', 1: 'I'}
{0: 'I', 1: 'X'}
{0: 'Y+', 1: 'X+'}
{0: 'I', 1: 'X'}
{0: 'Y+', 1: 'X-'}
{0: 'I', 1: 'X'}
{0: 'Y+', 1: 'Y+'}
{0: 'I', 1: 'X'}
{0: 'Y+', 1: 'Y-'}
{0: 'I', 1: 'X'}
{0: 'Y+', 1: 'Z+'}
{0: 'I', 1: 'X'}
{0: 'Y+', 1: 'Z-'}
{0: 'I', 1: 'X'}
{0: 'Y+', 1: 'I'}
{0: 'I', 1: 'X'}
{0: 'Y-', 1: 'X+'}
{0: 'I', 1: 'X'}
{0: 'Y-', 1: 'X-'}
{0: 'I', 1: 'X'}
{0: 'Y-', 1: 'Y+'}
{0: 'I', 1: 'X'}
{0: 'Y-', 1: 'Y-'}
{0: 'I', 1: 'X'}
{0: 'Y-', 1: 'Z+'}
{0: 'I', 1: 'X'}
{0: 'Y-', 1: 'Z-'}
{0: 'I', 1: 'X'}
{0: 'Y-', 1: '